In [18]:
from manim import *
from manim.utils.color import Colors
# from manim_fontawesome import *
from manim_fonts import *
from random import seed, shuffle
import numpy as np

%load_ext autoreload
%autoreload 2

from nextgen_01_defs import *
from nextgen_01_lib import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Probability via simulation

In [23]:
%%manim -v WARNING --progress_bar None NG_01_02_ProbViaSim


class NG_01_02_ProbViaSim(Scene):
    def construct(self):
        def create_person(marker=WELL_SYMB, color=NEG_COLOR):
            return VGroup(
                SVGMobject(
                    "noun-person-1492700.svg",
                    fill_color=color,
                ).set_z_index(0),
                patient_state_icon(
                    marker, BLACK,
                ).scale(0.75).set_z_index(0.5)
            )
        DOOR_OFFSET = LEFT * 4
        
        corridor = Rectangle(
            height=2, width=2,
            color=BLACK,
            fill_color=BLACK,
            fill_opacity=1.0
        ).shift(DOOR_OFFSET+LEFT*1.5).set_z_index(1)

        door = Rectangle(height=2, width=1).set_z_index(3).shift(DOOR_OFFSET)

        self.add(corridor)
        self.play(
            FadeIn(door),
        )
        chart = BarChart(
            values=[0,0,0,0],
            bar_names=[WELL_SYMB, WELL_SYMB, SICK_SYMB, SICK_SYMB],
            bar_colors=[NEG_COLOR, POS_COLOR, NEG_COLOR, POS_COLOR],
            y_range=[0, 1.0, 0.1],
            y_length=6,
            x_length=4,
            # x_axis_config={"font_size": 36},
        )
        chart.shift(4*RIGHT)
        self.play(
            FadeIn(chart)
        )
        count = np.array([0, 0, 0, 0])
        for i in range(20):
            if i==0:
                p = np.array([0.95, 0.8])
            elif i==1:
                p = np.array([0.94, 0.95])
            elif i==2:
                p = np.array([0.04, 0.95])
            else:
                p = np.random.random(2)
            if p[1]<0.9: # well
                symb = WELL_SYMB
                if p[0]>0.9: # pos
                    color = POS_COLOR
                    count[1] += 1
                else:
                    color = NEG_COLOR
                    count[0] += 1
            else:  # sick
                symb = SICK_SYMB
                if p[0]>0.1: # pos
                    color = POS_COLOR
                    count[3] += 1
                else:
                    color = NEG_COLOR
                    count[2] += 1
            person = create_person(
                symb, color
            ).shift(DOOR_OFFSET + LEFT*1.5)

            self.add(person)
            self.play(Succession(
                MoveAlongPath(
                    person,
                    Line(person.get_center(), RIGHT*3.5 + person.get_center())
                ),
                chart.animate.change_bar_values(count/count.sum()),
                FadeOut(person),
                run_time=3
            ))


In [ ]:
%%manim -v WARNING --progress_bar None NG_01_03_ProbViaSim

seed(18)

class NG_01_03_ProbViaSim(Scene):

    def construct(self):
        scale = 0.2
        POS_COLOR = BLUE
        NEG_COLOR = RED

        targetspace = Axes(
            y_range=[0, 1.0, 0.1],
            x_range=[0, 1.0, 0.1],
            y_length=4,
            x_length=4,
            # x_axis_config={"font_size": 36},
        )
        targetspace.shift(4*LEFT)
        self.add(targetspace)

        def as_rectangle_corners(bottom_left, top_right):
            return [
                (top_right[0], top_right[1]),
                (bottom_left[0], top_right[1]),
                (bottom_left[0], bottom_left[1]),
                (top_right[0], bottom_left[1]),
            ]
        
        def get_rectangle(bottom_left, top_right, color=POS_COLOR):
            polygon = Polygon(
                *[
                    targetspace.c2p(*i)
                    for i in as_rectangle_corners(
                        bottom_left, top_right
                    )
                ]
            )
            polygon.stroke_width = 1
            polygon.set_fill(color, opacity=0.3)
            polygon.set_stroke(color)
            return polygon

        def in_zone(zone, point):
            return (
                zone[0][0] <= point[0] < zone[1][0]
                and zone[0][1] <= point[1] < zone[1][1]
            )
        
        icon_scale = 0.6
        well_neg_zone = (0.0, 0.0), (0.9, 0.9)
        well_neg_rect = get_rectangle(*well_neg_zone, NEG_COLOR)
        self.add(well_neg_rect)
        self.add(patient_state_icon(WELL_SYMB, color=NEG_COLOR).scale(icon_scale).shift(well_neg_rect.get_center()))
        well_pos_zone = (0.9, 0.0), (1.0, 0.9)
        well_pos_rect = get_rectangle(*well_pos_zone, POS_COLOR)
        self.add(well_pos_rect)
        self.add(patient_state_icon(WELL_SYMB, color=POS_COLOR).scale(icon_scale).shift(well_pos_rect.get_center()))
        sick_neg_zone = (0.0, 0.9), (0.1, 1.0)
        sick_neg_rect = get_rectangle(*sick_neg_zone, NEG_COLOR)
        self.add(sick_neg_rect)
        self.add(patient_state_icon(SICK_SYMB, color=NEG_COLOR).scale(icon_scale).shift(sick_neg_rect.get_center()))
        sick_pos_zone = (0.1, 0.9), (1.0, 1.0)
        sick_pos_rect = get_rectangle(*sick_pos_zone, POS_COLOR)
        self.add(sick_pos_rect)
        self.add(patient_state_icon(SICK_SYMB, color=POS_COLOR).scale(icon_scale).shift(sick_pos_rect.get_center()))

        zones = [well_neg_zone, well_pos_zone, sick_neg_zone, sick_pos_zone]
        count = np.array([0, 0, 0, 0])

        def zone(point):
            for i, zone in enumerate(zones):
                if in_zone(zone, point):
                    return i, zone
            raise ValueError("Point not in any zone")

        chart = BarChart(
            values=[0,0,0,0],
            bar_names=[ WELL_SYMB, WELL_SYMB, SICK_SYMB, SICK_SYMB],
            bar_colors=[NEG_COLOR, POS_COLOR, NEG_COLOR, POS_COLOR],
            y_range=[0, 1.0, 0.1],
            y_length=6,
            x_length=4,
            # x_axis_config={"font_size": 36},
        )
        chart.shift(4*RIGHT)

        ## various attempts to animate te bar labels, all failed
        # c_bar_lbls = chart.get_bar_labels(font_size=24, 
        #     label_constructor=lambda f: DecimalNumber(float(f), num_decimal_places=2))    
        # for i, lab in enumerate(c_bar_lbls):
        #     # lab.add_updater(lambda l: l.set_value(chart.values[i]))
        #     lab.add_updater(lambda l: l.become(chart.bar_labels[i]))
        # self.add(chart, c_bar_lbls)
        # c_bar_lbls.add_updater(lambda l: l.become(chart.bar_labels))
        self.wait(1)
        for n_points_now, dur in [(1,1.0),] *10 + [(2*k , 1/k) for k in range(1, 25)]: 
            for step in range(n_points_now):
                # override first few points so the bars start out even-ish
                if count.sum() == 0:
                    p = np.array([0.45, 0.95])
                elif count.sum() == 1:
                    p = np.array([0.12, 0.8])
                elif count.sum() == 2:
                    p = np.array([0.03, 0.97])
                elif count.sum() == 3:
                    p = np.array([0.93, 0.2])
                elif count.sum() == 4:
                    p = np.array([0.03, 0.97])
                else:
                    p = np.random.random(2)
                self.add(
                    Dot(
                        targetspace.c2p(*p)
                    ))
                i, z = zone(p)
                count[i] += 1
                if count.sum()>13: break
                self.play(
                    chart.animate.change_bar_values(count/count.sum()),
                    # Transform(
                    #     c_bar_lbls,
                    #     chart.bar_labels),
                    run_time=dur)


